### Load the libraries

In [5]:
l1 = ['abia', 'ogun']

d = {i:l1.index(i)+1 for i in l1}
d

{'abia': 1, 'ogun': 2}

In [4]:
l1.index('ogun')
for i in l1:
    print(i)

abia
ogun


In [1]:
import pandas as pd
import os
import glob
import sqlitecloud

### Data Preprocessing
#### Load the data files - csv

In [2]:
# Explain cwd and glob
path = os.getcwd()
csv_files22 = glob.glob(os.path.join(path, "data22", "*.csv"))

csv_files = glob.glob(os.path.join(path, "data", "*.csv"))

In [3]:
# csv_files

#### Convert each csv to a DataFrame and add it to a list of dataframes

In [4]:
dfs = []

for file in csv_files:
    
    dfs.append(pd.read_csv(file))

for file in csv_files22:
    
    dfs.append(pd.read_csv(file))

### _Data validation_

Inspect all dataframes for consistency of datatypes and column names, and finally merged into one big dataframe.

In [51]:
# dfs1 = []
# for df in dfs:
#     df1 = df.sample(frac=1)
#     df1 = df1.head(18000)
#     dfs1.append(df1)

In [5]:
len(dfs)

24

The results revealed that the column names and data types of the 12 data files were consistent.

To further investigate and clean the data, lets merge the data files into one dataframe.

In [6]:
trip_data = pd.concat(dfs)
# trip_data.to_csv("divvy-data.csv", index=False)

#### _Data Cleaning_

In this section, we will check and remove duplicate values if any, handle missing values, and rename some columns with more descriptive names.

#### _Check for duplicate records_

In [7]:
# trip_data.duplicated().sum()
trip_data.shape

(11387594, 13)

In [54]:
trip_data["started"] = pd.to_datetime(trip_data["started_at"])
trip_data['quarter'] = trip_data['started'].dt.quarter
trip_data['quarter'] = "qtr " + trip_data['quarter'].astype(str)
trip_data['month'] = trip_data['started'].dt.strftime('%B')
trip_data['month_num'] = trip_data['started'].dt.month

trip_data.drop(columns=['started'], inplace=True)

trip_data.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,quarter,month,month_num
36299,A4BBAFE8703ACBDC,electric_bike,2022-01-19 09:51:33,2022-01-19 09:55:55,Columbus Dr & Randolph St,13263,Daley Center Plaza,TA1306000010,41.884563,-87.619481,41.884241,-87.629634,member,qtr 1,January,1
73314,A8FAAC342B46D1C0,classic_bike,2022-01-19 06:37:10,2022-01-19 06:44:19,Carpenter St & Huron St,13196,Rush St & Hubbard St,KA1503000044,41.894556,-87.653449,41.890173,-87.626185,member,qtr 1,January,1
41157,5E718BFD88FF318F,classic_bike,2022-01-07 17:29:48,2022-01-07 17:33:10,Racine Ave & Wrightwood Ave,TA1309000059,Halsted St & Wrightwood Ave,TA1309000061,41.928887,-87.658971,41.929143,-87.649077,member,qtr 1,January,1
53121,847D2AF5E0237109,classic_bike,2022-01-25 09:06:41,2022-01-25 09:14:37,Michigan Ave & Jackson Blvd,TA1309000002,Clinton St & Jackson Blvd,638,41.877850,-87.624080,41.878317,-87.640981,member,qtr 1,January,1
87617,38966D467D0EDD76,classic_bike,2022-01-13 16:17:11,2022-01-13 16:25:56,Dearborn St & Erie St,13045,Kingsbury St & Kinzie St,KA1503000043,41.893992,-87.629318,41.889177,-87.638506,member,qtr 1,January,1


In [59]:
trip_data['quarter'].str.split(" ").str[1]
"kdjfkld".

36299     1
73314     1
41157     1
53121     1
87617     1
         ..
91570     4
172088    4
29946     4
37381     4
123211    4
Name: quarter, Length: 216000, dtype: object

In [7]:
trip_data.sample()

array(['January', 'February', 'March', 'April', 'May', 'June', 'July',
       'August', 'September', 'October', 'November', 'December'],
      dtype=object)

In [55]:
# conn = sqlitecloud.connect("sqlitecloud://cc6om9hcik.sqlite.cloud:8860?apikey=Xy45kp2O1gdyjHVeThtZjdu833oIZvpWkOYqlWRaDik")

# db_name = "divvydb"

# conn.execute(f"USE DATABASE {db_name}")

# conn.executemany("DROP TABLE IF EXISTS ?", [("divvy_bike22",)])
import sqlite3

conn = sqlite3.connect('divvydb.sqlite')
trip_data.to_sql('divvy_bike22', conn, index=False)
conn.close()

In [43]:
conn = sqlite3.connect('divvydb.sqlite')
df = pd.read_sql_query("select * from divvy_bike_share where month = 'January'", conn)

df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,quarter,month
0,F96D5A74A3E41399,electric_bike,2023-01-21 20:05:42,2023-01-21 20:16:33,Lincoln Ave & Fullerton Ave,TA1309000058,Hampden Ct & Diversey Ave,202480.0,41.924074,-87.646278,41.930000,-87.640000,member,qtr 1,January
1,13CB7EB698CEDB88,classic_bike,2023-01-10 15:37:36,2023-01-10 15:46:05,Kimbark Ave & 53rd St,TA1309000037,Greenwood Ave & 47th St,TA1308000002,41.799568,-87.594747,41.809835,-87.599383,member,qtr 1,January
2,BD88A2E670661CE5,electric_bike,2023-01-02 07:51:57,2023-01-02 08:05:11,Western Ave & Lunt Ave,RP-005,Valli Produce - Evanston Plaza,599,42.008571,-87.690483,42.039742,-87.699413,casual,qtr 1,January
3,C90792D034FED968,classic_bike,2023-01-22 10:52:58,2023-01-22 11:01:44,Kimbark Ave & 53rd St,TA1309000037,Greenwood Ave & 47th St,TA1308000002,41.799568,-87.594747,41.809835,-87.599383,member,qtr 1,January
4,3397017529188E8A,classic_bike,2023-01-12 13:58:01,2023-01-12 14:13:20,Kimbark Ave & 53rd St,TA1309000037,Greenwood Ave & 47th St,TA1308000002,41.799568,-87.594747,41.809835,-87.599383,member,qtr 1,January


This result shows that there are no duplicate values, each row of the trip record represent a unique trip.

Let's determine the total number of trips recorded between January and December 2023

### _What is the total number of trips recorded_

In [14]:
print(trip_data.shape)

total_trips, _ = trip_data.shape # you can also do total_trips = trip_data.shape[0]

(5719877, 13)


The result shows that between January and December 2023, a total 5.7 million trips were recorded by Cyclistic bikes.

Next, lets rename the columns ride_id and rideable_type with a more descriptive names - trip_id and bike_type respectively.

In [15]:
trip_data.rename(columns={"ride_id": "trip_id", "rideable_type": "bike_type"}, inplace=True)

trip_data.columns

Index(['trip_id', 'bike_type', 'started_at', 'ended_at', 'start_station_name',
       'start_station_id', 'end_station_name', 'end_station_id', 'start_lat',
       'start_lng', 'end_lat', 'end_lng', 'member_casual'],
      dtype='object')

### _Check for missing values_

In [16]:
trip_data.isnull().sum()

trip_id                    0
bike_type                  0
started_at                 0
ended_at                   0
start_station_name    875716
start_station_id      875848
end_station_name      929202
end_station_id        929343
start_lat                  0
start_lng                  0
end_lat                 6990
end_lng                 6990
member_casual              0
dtype: int64

The columns with missing values above reveals that there are 929,202 record with no start station or end station name and id. Hence, it will be difficult to track the routes of these trips. These records will be removed as their route information are inconclusive. However, this observation will be communicated to the team lead for further investigation.

Remove records without start or end station details

In [17]:
trip_data = trip_data.dropna(subset=["start_station_id", "end_station_id", "end_lat", "end_lng"])

trip_data.isnull().sum()

trip_id               0
bike_type             0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
dtype: int64

After cleaning the missing and empty values, it time to perform EDA to answer questions for further investigation.

### _Overview of the data_

In [18]:
trip_data.describe(include='all')

,trip_id,bike_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
count,4331707,4331707,4331707,4331707,4331707,4331707,4331707,4331707,4.331707e+06,4.331707e+06,4.331707e+06,4.331707e+06,4331707
unique,4331707,3,3790408,3801204,1534,1468,1557,1483,NaN,NaN,NaN,NaN,2
top,F96D5A74A3E41399,classic_bike,2023-05-19 16:18:05,2023-12-08 09:54:28,Streeter Dr & Grand Ave,13022,Streeter Dr & Grand Ave,13022,NaN,NaN,NaN,NaN,member
freq,1,2690744,7,18,59614,59614,61713,61713,NaN,NaN,NaN,NaN,2799950
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.190095e+01,-8.764466e+01,4.190137e+01,-8.764487e+01,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.301374e-02,2.490093e-02,5.549544e-02,7.712000e-02,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.164850e+01,-8.784409e+01,0.000000e+00,-8.784406e+01,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.188090e+01,-8.765766e+01,4.188096e+01,-8.765842e+01,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.189636e+01,-8.764170e+01,4.189662e+01,-8.764182e+01,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.192677e+01,-8.762858e+01,4.192877e+01,-8.762858e+01,NaN


### Exploratory Data Analysis

At this stage, I will be using specific questions to guide the process, and analysis the data to provide answers.

#### What is the total number of valid trips?

In [19]:
num_of_valid_trips = (trip_data.shape)[0]

num_of_valid_trips

4331707

There are a total of 4.33 million valid trips record by Cyclistic bikes in 2023.

#### What is the number of trips with missing details?

In [20]:
num_of_invalid_trips = total_trips - num_of_valid_trips

num_of_invalid_trips

1388170

#### What percentage of the trip records were invalid?

In [21]:
percent_of_invalid_trips = (num_of_invalid_trips/total_trips)*100

round(percent_of_invalid_trips,1)

24.3

The above results indicates that a total of 1.35 million records, which form 24.3% of the overrall trip records for the period were incorrectly captured.

The ideal next thing to do will be to report this discovery to the management throught the team lead for further investigation on what went wrong with the incorrectly captured data, before continuing with the analysis.

### Create new columns

In order to better understand and make sense the trip data, we need to know how long a trip took per day, so two new columns (trip_duration and week_day) will be created.

In [22]:
trip_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4331707 entries, 0 to 224072
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   trip_id             object 
 1   bike_type           object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 462.7+ MB


Convert date related columns to datetime objects

In [23]:
trip_data["started_at"] = pd.to_datetime(trip_data["started_at"])

trip_data["ended_at"] = pd.to_datetime(trip_data["ended_at"])

trip_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4331707 entries, 0 to 224072
Data columns (total 13 columns):
 #   Column              Dtype         
---  ------              -----         
 0   trip_id             object        
 1   bike_type           object        
 2   started_at          datetime64[ns]
 3   ended_at            datetime64[ns]
 4   start_station_name  object        
 5   start_station_id    object        
 6   end_station_name    object        
 7   end_station_id      object        
 8   start_lat           float64       
 9   start_lng           float64       
 10  end_lat             float64       
 11  end_lng             float64       
 12  member_casual       object        
dtypes: datetime64[ns](2), float64(4), object(7)
memory usage: 462.7+ MB


Create a week_day column

In [24]:
trip_data['week_day'] = trip_data["started_at"].dt.day_name() # .strftime('%a') .strftime('%A')

trip_data.week_day.unique()

array(['Saturday', 'Tuesday', 'Monday', 'Sunday', 'Thursday', 'Wednesday',
       'Friday'], dtype=object)

#### What is the average trip duration?

In [25]:
trip_data['trip_duration'] = round((trip_data["ended_at"] - trip_data["started_at"]).dt.total_seconds()/60,1)

round(trip_data['trip_duration'].mean(),2)

15.95

#### Task: Convert this time to a proper time format i.e  `minute:seconds` format

In [26]:
trip_data['trip_dur'] = (trip_data['trip_duration']//1).astype(int).astype(str)+':'+(((trip_data['trip_duration']%1)*60)//1).astype(int).astype(str)

trip_data.head()

,trip_id,bike_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,week_day,trip_duration,trip_dur
0,F96D5A74A3E41399,electric_bike,2023-01-21 20:05:42,2023-01-21 20:16:33,Lincoln Ave & Fullerton Ave,TA1309000058,Hampden Ct & Diversey Ave,202480.0,41.924074,-87.646278,41.930000,-87.640000,member,Saturday,10.8,10:48
1,13CB7EB698CEDB88,classic_bike,2023-01-10 15:37:36,2023-01-10 15:46:05,Kimbark Ave & 53rd St,TA1309000037,Greenwood Ave & 47th St,TA1308000002,41.799568,-87.594747,41.809835,-87.599383,member,Tuesday,8.5,8:30
2,BD88A2E670661CE5,electric_bike,2023-01-02 07:51:57,2023-01-02 08:05:11,Western Ave & Lunt Ave,RP-005,Valli Produce - Evanston Plaza,599,42.008571,-87.690483,42.039742,-87.699413,casual,Monday,13.2,13:11
3,C90792D034FED968,classic_bike,2023-01-22 10:52:58,2023-01-22 11:01:44,Kimbark Ave & 53rd St,TA1309000037,Greenwood Ave & 47th St,TA1308000002,41.799568,-87.594747,41.809835,-87.599383,member,Sunday,8.8,8:48
4,3397017529188E8A,classic_bike,2023-01-12 13:58:01,2023-01-12 14:13:20,Kimbark Ave & 53rd St,TA1309000037,Greenwood Ave & 47th St,TA1308000002,41.799568,-87.594747,41.809835,-87.599383,member,Thursday,15.3,15:18


In [27]:
trip_duration = str(int(round(trip_data['trip_duration'].mean(),2)//1))+':'+ str(int(round((round(trip_data['trip_duration'].mean(),2)%1)*60,1)))

Here, we can see that the average trip duration is 15:57 per trip.

In [28]:
import datetime
time = datetime.datetime.strptime('15:57', '%M:%S')

In [29]:
time.strftime("%M:%S")

'15:57'

The plotly Python library is an interactive, open-source plotting library that supports over 40 unique chart types covering a wide range of statistical, financial, geographic, scientific, and 3-dimensional use-cases.

In [30]:
import plotly.graph_objects as go
import plotly.express as px

def indicator(value, title, suffix=None):
    fig = go.Figure()

    fig.add_trace(
        go.Indicator(
            value=value,
            number={
                "font.size": 30,
                "font.color": "black",
                # "prefix":"$", 
                "suffix":suffix
            },
            title={
                "text": title,
                "font": {"size": 15, 'color':'black'},
            },
        )
    )

    # fig.update_xaxes(visible=False, fixedrange=True)
    # fig.update_yaxes(visible=False, fixedrange=True)
    fig.update_layout(
        height=100,
        width=250
    )

    return fig

In [31]:
indicator(num_of_valid_trips, 'Total Trips')

In [32]:
indicator(0, 'Average trip duration', ":"+trip_duration)

#### What is the proportion of user types?

In [33]:
rider_prop = trip_data.groupby('member_casual', as_index=False)["member_casual"].value_counts()

In [34]:
def donut_chart(df, title):
    fig = px.pie(
        df, values='count', 
        names='member_casual',
        title=title,
        hole=0.6,
        color='member_casual',
        labels={'count':'Total Trips',
                'member_casual':'Membership Type'},
        color_discrete_map={
            'member':'#8BC7F7',
            'casual': '#D5EA67'
        }
    )
    fig.update_traces(textposition='outside', textinfo='value+percent')
    fig.update_layout(
        height=260,
        width=300,
        showlegend=True,
    )
    return fig

In [35]:
donut_chart(rider_prop, 'Proportion of Membership')

> -  Casual riders made 35.4% of the total trips recorded.
> -  Annual members made 64.6% of the recorded trips

#### What is the most used bike types by membership types?

In [36]:
member_by_bike = trip_data.groupby(['member_casual','bike_type'], as_index=False)["member_casual"].value_counts()

member_by_bike['tot_trip'] = member_by_bike['count'].apply(lambda x: '{0:1.2f}M'.format(x/1000000))

member_by_bike = member_by_bike.sort_values(by='count')

member_by_bike

,member_casual,bike_type,count,tot_trip
1,casual,docked_bike,76124,0.08M
2,casual,electric_bike,582608,0.58M
0,casual,classic_bike,873025,0.87M
4,member,electric_bike,982231,0.98M
3,member,classic_bike,1817719,1.82M


In [37]:
def plot(df, x, y, orientation=None, title=None, color=None, text=None, labels=None, color_discrete_map=None, hover_data=None, hover_name=None):
    # if met is None:
    #     text = text
    # else:
    #     text = df[text].apply(lambda x: '{0:1.2f}M'.format(x/met))
    fig = px.bar(
        df,
        x=y,
        y=x,
        orientation=orientation,
        title=title,
        color=color,
        text=text,
        barmode='group',
        labels=labels,
        color_discrete_map=color_discrete_map,
        hover_data=hover_data,
        hover_name=hover_name
        
    )
    
    fig.update_xaxes(visible=True, title="", fixedrange=True)
    fig.update_yaxes(visible=True, title="", fixedrange=True, showticklabels=True)
    fig.update_layout(
        title=title,
        showlegend=False,
        paper_bgcolor='rgb(248, 248, 255)',
        plot_bgcolor='rgb(248, 248, 255)',
        # width=500,
        # template="plotly_white"
    )

    return fig

In [38]:
color_discrete_map = {
    'member':'#8BC7F7',
    'casual': '#D5EA67'
}

hover_data={
    'tot_trip': False,
    # 'count': False
}

labels={
    'count':'Total Trips',
    'member_casual':'Membership Type',
    'bike_type': 'Bike Type',
    # text: 'Total Trips'
}

plot(member_by_bike, 'bike_type', 'count', title='Total trips by members per bike', orientation='h', color='member_casual', text='tot_trip', color_discrete_map=color_discrete_map, hover_data=hover_data, labels=labels)

> -  Most used Bike by both user groups is Classic bikes
> -  Docked bikes are only used by casual riders, and covered 4.9% of their total trips.

In [39]:
member_by_dur = trip_data.groupby(['member_casual','bike_type'], as_index=False)["trip_duration"].mean()

member_by_dur = member_by_dur.sort_values(by='trip_duration')
member_by_dur['trip_dur'] = (member_by_dur['trip_duration']//1).astype(int).astype(str)+':'+(round((member_by_dur['trip_duration']%1)*60,1)//1).astype(int).astype(str)

member_by_dur

,member_casual,bike_type,trip_duration,trip_dur
4,member,electric_bike,10.526780,10:31
3,member,classic_bike,12.997680,12:59
2,casual,electric_bike,14.663562,14:39
0,casual,classic_bike,25.750286,25:45
1,casual,docked_bike,53.976109,53:58


In [40]:
mins = (member_by_dur['trip_duration']//1).astype(int).astype(str)

In [41]:
sec = (round((member_by_dur['trip_duration']%1)*60,1)//1).astype(int).astype(str)

In [42]:
(mins+':'+sec)

4    10:31
3    12:59
2    14:39
0    25:45
1    53:58
Name: trip_duration, dtype: object

In [43]:
labels={
    'member_casual':'Membership Type',
    'bike_type': 'Bike Type',
    'trip_dur': 'Trip Duration'
}

hover_data={
    'trip_duration': False
}

plot(member_by_dur, 'bike_type', 'trip_duration', title='Trip Duration by User type per Bike type', color='member_casual', text='trip_dur', color_discrete_map=color_discrete_map, labels=labels, hover_data=hover_data, orientation='h')

> -  Casual riders travel longer time per trip on average when compared with annual members
> -  Docked bike users traveled the longest trip duration on average.

#### Which week days does casual riders record highest number of trips?

In [44]:
trip_data['day_num'] = trip_data['started_at'].dt.dayofweek
trip_data['day_num'] = trip_data['day_num'].replace([6,0,1,2,3,4,5],[1,2,3,4,5,6,7])

member_by_trip = trip_data.groupby(['member_casual','week_day','day_num'], as_index=False)["member_casual"].value_counts()

member_by_trip = member_by_trip.sort_values(by=['member_casual','day_num'])

member_by_trip

,member_casual,week_day,day_num,count
3,casual,Sunday,1,254761
1,casual,Monday,2,175401
5,casual,Tuesday,3,181537
6,casual,Wednesday,4,183092
4,casual,Thursday,5,198931
0,casual,Friday,6,227869
2,casual,Saturday,7,310166
10,member,Sunday,1,307870
8,member,Monday,2,386697
12,member,Tuesday,3,448850


In [45]:
def lineplot(df, x, y, title, color=None, text=None, line=True, duration=None):

    if line:
        fig = px.line(
            df,
            x=x,
            y=y,
            title=title,
            color=color,
            # text=text,
            category_orders={'week_day': ['Sunday','Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']},
            color_discrete_map={
                'member':'#8BC7F7',
                'casual': '#D5EA67'
            },
            labels={'count':'Total Trips',
                'member_casual':'Membership Type',
                'week_day': 'Week Day',
                text: 'Trip Duration'
            },
            hover_data={text: True,
                duration: False  
            }
        )
        fig.update_traces(textposition="top center", fill="tozeroy")
    else: 
        fig = px.bar(
            df,
            x=x,
            y=y,
            orientation="h",
            title=title,
            color=color,
            text=x,
            barmode='group',
            
        )
    
    fig.update_xaxes(visible=True, title="", fixedrange=True)
    fig.update_yaxes(visible=True, title="", fixedrange=True, showticklabels=True)
    fig.update_layout(
        paper_bgcolor='rgb(248, 248, 255)',
        plot_bgcolor='rgb(248, 248, 255)',
        title=title,
        height=350,
        showlegend=False,
        # width=500,
        # template="plotly_white"
    )

    return fig

In [46]:
lineplot(member_by_trip, 'week_day', 'count', 'Total trips per day per members', 'member_casual')

In [47]:
member_by_duration = trip_data.groupby(['member_casual','week_day','day_num'], as_index=False)["trip_duration"].mean()

member_by_duration = member_by_duration.sort_values(by=['member_casual','day_num'])

member_by_duration['trip_dur'] = (member_by_duration['trip_duration']//1).astype(int).astype(str)+':'+(round((member_by_duration['trip_duration']%1)*60,1)//1).astype(int).astype(str)

member_by_duration

,member_casual,week_day,day_num,trip_duration,trip_dur
3,casual,Sunday,1,26.559772,26:33
1,casual,Monday,2,22.534096,22:32
5,casual,Tuesday,3,20.511711,20:30
6,casual,Wednesday,4,19.598082,19:35
4,casual,Thursday,5,19.997074,19:59
0,casual,Friday,6,22.317905,22:19
2,casual,Saturday,7,25.915957,25:55
10,member,Sunday,1,13.610619,13:36
8,member,Monday,2,11.549542,11:33
12,member,Tuesday,3,11.648062,11:38


In [48]:
lineplot(member_by_duration, 'week_day', 'trip_duration', 'Avg. Trip duration per day', 'member_casual', text='trip_dur', duration='trip_duration')

#### What are the most used routes by membership types?

In [49]:
trip_data['route'] = trip_data['start_station_name']+'_to_'+trip_data['end_station_name']

trip_data['route']

0         Lincoln Ave & Fullerton Ave_to_Hampden Ct & Di...
1          Kimbark Ave & 53rd St_to_Greenwood Ave & 47th St
2         Western Ave & Lunt Ave_to_Valli Produce - Evan...
3          Kimbark Ave & 53rd St_to_Greenwood Ave & 47th St
4          Kimbark Ave & 53rd St_to_Greenwood Ave & 47th St
                                ...                        
224068      900 W Harrison St_to_Racine Ave & Congress Pkwy
224069      900 W Harrison St_to_Racine Ave & Congress Pkwy
224070      900 W Harrison St_to_Racine Ave & Congress Pkwy
224071       Damen Ave & Madison St_to_Morgan St & Lake St*
224072      900 W Harrison St_to_Racine Ave & Congress Pkwy
Name: route, Length: 4331707, dtype: object

In [50]:
casual = trip_data.loc[trip_data['member_casual'] == 'casual',]
casual.head()

,trip_id,bike_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,week_day,trip_duration,trip_dur,day_num,route
2,BD88A2E670661CE5,electric_bike,2023-01-02 07:51:57,2023-01-02 08:05:11,Western Ave & Lunt Ave,RP-005,Valli Produce - Evanston Plaza,599,42.008571,-87.690483,42.039742,-87.699413,casual,Monday,13.2,13:11,2,Western Ave & Lunt Ave_to_Valli Produce - Evan...
13,9DC70E5EE9D6A93F,electric_bike,2023-01-03 20:25:53,2023-01-03 20:35:50,Broadway & Waveland Ave,13325,Hampden Ct & Diversey Ave,202480.0,41.949106,-87.648628,41.930000,-87.640000,casual,Tuesday,10.0,10:0,3,Broadway & Waveland Ave_to_Hampden Ct & Divers...
17,689D537E5D034DA8,electric_bike,2023-01-05 17:28:08,2023-01-05 17:43:24,Western Ave & Lunt Ave,RP-005,Valli Produce - Evanston Plaza,599,42.008614,-87.690523,42.039742,-87.699413,casual,Thursday,15.3,15:18,5,Western Ave & Lunt Ave_to_Valli Produce - Evan...
22,A5CBC7142CA8FAD8,classic_bike,2023-01-02 12:45:48,2023-01-02 12:53:06,Clark St & Berwyn Ave,KA1504000146,Clark St & Elmdale Ave,KA1504000148,41.978031,-87.668565,41.990860,-87.669724,casual,Monday,7.3,7:17,2,Clark St & Berwyn Ave_to_Clark St & Elmdale Ave
24,2B8E0781ED90C27C,electric_bike,2023-01-11 17:03:25,2023-01-11 17:13:34,Avondale Ave & Irving Park Rd,15624,Campbell Ave & Irving Park Rd,439,41.953178,-87.731948,41.950000,-87.690000,casual,Wednesday,10.2,10:11,4,Avondale Ave & Irving Park Rd_to_Campbell Ave ...


In [51]:
casual = trip_data.loc[trip_data['member_casual'] == 'casual',]
casual = casual.groupby('route', as_index=False)['route'].value_counts()
casual = casual.sort_values(by='count', ascending=False)

casual = casual.head()

In [52]:
casual = casual.sort_values(by='count')

In [53]:
labels={
    'member_casual':'Membership Type',
    'bike_type': 'Bike Type',
    'trip_dur': 'Trip Duration'
}

hover_data={
    'trip_duration': False
}

colors = ['rgba(213,234,103,0.3)',]*5
colors[4] = 'rgba(213,234,103,1)'

def casual_chart(x, y, marker_color=None, orientation=None, text=None, title=None):
    fig = go.Figure()

    fig.add_trace(
        go.Bar(
            x=x,
            y=y,
            marker_color=marker_color,
            orientation=orientation,
            text=text.apply(lambda x: '{0:1.1f}k'.format(x/1000))

        )
    )
    fig.update_layout(
        title_text=title,
        paper_bgcolor='rgb(248, 248, 255)',
        plot_bgcolor='rgb(248, 248, 255)',
        width=700,
    )
    return fig

# plot(casual, 'route', 'count', title='Top 5 Routes of Casual riders Number trips', color=colors,orientation='h')

In [54]:
casual_chart(casual['count'], casual['route'], marker_color=colors, orientation='h', text=casual['count'], title='Top 5 Routes of Casual riders Number trips')

> -  Most used route by Casual riders is a round trip - from and to Streeter Dr & Ave.
> -  Majority of trips by Casual riders are round trips.

In [55]:
member = trip_data.loc[trip_data['member_casual'] == 'member',]
member = member.groupby('route', as_index=False)['route'].value_counts()
member = member.sort_values(by='count', ascending=False)

member = member.head()
member = member.sort_values(by='count')

In [56]:
colors = ['rgba(139,199,247,0.3)',]*5
colors[4] = 'rgba(139,199,247,1)'
colors[3] = 'rgba(139,199,247,1)'
casual_chart(member['count'], member['route'], marker_color=colors, orientation='h', text=member['count'], title='Top 5 Routes of Annual Members Number trips')

> -  Most used route by annual members are Ellis Ave & 60th St to University Ave & 57th St and Calumet Ave & 33rd St_to_State St & 33rd St
> -  Trips by Annual Member are mostly one way trip per time.

In [57]:
# trip_data.reset_index()
trip_data.to_csv("divvy-data.csv", index=False)

In [60]:
df = pd.read_csv("divvy-data.csv")
df.shape

(5719877, 13)